In [ ]:
!pip install transformers
!pip install gradio
!pip install sentencepiece

In [ ]:
import transformers
import torch
from time import time
import gradio as gr
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [ ]:
def load_model(model_name):
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
    if model_name == 't5-small':
        tokenizer_name = "t5-small"
        path = "arver/t5-small-boolean-qgen"
    elif model_name == 't5-base-finetuned':
        tokenizer_name = "t5-base"
        path = "arver/t5-base-boolean-qgen-direct-finetune"
    elif model_name == 't5-base-qgen-finetuned':
        tokenizer_name = "t5-base"
        path = "arver/t5-base-boolean-qgen_pretrained-finetuned"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    model = T5ForConditionalGeneration.from_pretrained(path)
    return tokenizer, model.to(device)

In [ ]:
def process_one_sample(tokenizer, context, answer, device, max_len=256, truncation=True, pad_to_max_len=True):
  tokenizer_input = "answer: %s context: %s" % (answer, context)
  tokenized_input = tokenizer(tokenizer_input, max_length=max_len, truncation=truncation, pad_to_max_length=pad_to_max_len, return_tensors='pt')
  return tokenized_input['input_ids'].to(device), tokenized_input['attention_mask'].to(device)

In [ ]:
def beam_search_decoding (tokenizer, model, inp_ids,attn_mask, num_beams=5, num_return_sequences=2):
  beam_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               num_beams=num_beams,
                               num_return_sequences=num_return_sequences,
                               no_repeat_ngram_size=2,
                               early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True) for out in
               beam_output]
  return [Question.strip().capitalize() for Question in Questions]

In [ ]:
def predict_question(model_name, context, answer, num_beams=5, num_return_sequences=2):
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
    tokenizer, model = load_model(model_name)
    input_ids, attention_mask = process_one_sample(tokenizer, context, answer, device)
    start = time()
    questions = beam_search_decoding(tokenizer, model, input_ids, attention_mask, num_beams, num_return_sequences)
    end = time()
    output = ""
    c = 1
    for question in questions:
        output += str(c) + ". " + question + "\n"
        c += 1
    output += "\nInference time: " + str((end-start)*1000) + " ms"
    return output

In [ ]:
sample_context_true = "The states that have legislatively adopted stand-your-ground laws are Alabama, Alaska, Arizona, Florida, Georgia, Idaho, Indiana, Iowa, Kansas, Kentucky, Louisiana, Michigan, Mississippi, Missouri, Montana, Nevada, New Hampshire, North Carolina, Oklahoma, Pennsylvania, South Carolina, South Dakota, Tennessee, Texas, Utah, West Virginia, and Wyoming."
sample_context_false = "In geometric measurements, length is the most extended dimension of an object. In the International System of Quantities, length is any quantity with dimension distance. In other contexts, length is a measured dimension of an object. Length may be distinguished from height, which is vertical extent, and width or breadth, which are the distance from side to side, measuring across the object at right angles to the length. For example, it is possible to cut a length of wire shorter than the wire's width. In most systems of measurement, the unit of length is a base unit, from which other units are derived."
generate_questions_ui = gr.Interface(
    predict_question,
    inputs=[
        gr.Dropdown(
            ["t5-small", "t5-base-finetuned", "t5-base-qgen-finetuned"], label="Select model", info="t5-small version finetuned on boolq, t5-base version finetuned on boolq, t5 base version pretrained on qgen data then finetuned on boolq"
        ),
        gr.Textbox(label="Context"),
        gr.Dropdown(
            ["TRUE", "FALSE"], label="Answer", info="select TRUE if answer to question generated should be true, else select FALSE"
        ),
        gr.Slider(3, 10, value=5, label="Number of beams", step=1, info="Choose betwen 3 and 10, leave at 5 if not sure"),
        gr.Slider(1, 5, value=5, label="Number of questions generated", step=1, info="Choose betwen 1 and 5, leave at 2 if not sure")
    ],
    outputs=gr.Textbox(label="Questions generated"),
    examples=[
        ["t5-base-qgen-finetuned", sample_context_true, "TRUE", 5, 1],
        ["t5-base-qgen-finetuned", sample_context_false, "FALSE", 10, 1]
    ],
)

In [ ]:
generate_questions_ui.close()

In [ ]:
generate_questions_ui.launch(share=True, debug=True)